In [3]:
# pip install gym
import gym

import numpy as np
import random
import os
import collections

import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

mpl.rc('animation', html='jshtml')

from array2gif import write_gif

### OpenAI Gym. Маятник на тележке - Cartpole

![cartpole](https://www.gymlibrary.dev/_images/cart_pole.gif)

<font size="3">
<p>Эта среда - одна из систем в <a href='https://www.gymlibrary.dev/environments/classic_control/index.html'>Classic Control environments</a>.</p>
<div class="table-wrapper colwidths-auto docutils container">
<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p></p></th>
<th class="head"><p></p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>Пространство действий</p></td>
<td><p>Discrete(2)</p></td>
</tr>
<tr class="row-odd"><td><p>Размер вектора состояния</p></td>
<td><p>(4,)</p></td>
</tr>
<tr class="row-even"><td><p>Верхние границы</p></td>
<td><p>[4.8   inf 0.42  inf]</p></td>
</tr>
<tr class="row-odd"><td><p>Нижние границы</p></td>
<td><p>[-4.8   -inf -0.42  -inf]</p></td>
</tr>
<tr class="row-even"><td><p>Импорт</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">gym.make(&quot;CartPole-v1&quot;)</span></code></p></td>
</tr>
</tbody>
</table>
</font>

In [ ]:
env = gym.make('CartPole-v1')

### Состояния

<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p>№</p></th>
<th class="head"><p>Переменная</p></th>
<th class="head"><p>Min</p></th>
<th class="head"><p>Max</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>0</p></td>
<td><p>Положение тележки</p></td>
<td><p>-4.8</p></td>
<td><p>4.8</p></td>
</tr>
<tr class="row-odd"><td><p>1</p></td>
<td><p>Скорость тележки</p></td>
<td><p>-Inf</p></td>
<td><p>Inf</p></td>
</tr>
<tr class="row-even"><td><p>2</p></td>
<td><p>Угол маятника с вертикалью</p></td>
<td><p>~ -0.418 rad (-24°)</p></td>
<td><p>~ 0.418 rad (24°)</p></td>
</tr>
<tr class="row-odd"><td><p>3</p></td>
<td><p>Угловая скорость маятника</p></td>
<td><p>-Inf</p></td>
<td><p>Inf</p></td>
</tr>
</tbody>
</table>

In [ ]:
env.observation_space.shape[0]

In [ ]:
obs = env.reset()
obs

In [ ]:
Obs = collections.namedtuple("CartPole_obs", "x v theta omega")
Obs(*obs)

In [ ]:
img = env.render(mode="rgb_array")
plt.imshow(img)

### Начальные условия

Все переменные векора состояний инициализируются случайными величинами равномерно распределёнными в интервале (-0.05, 0.05)


### Начало эпизода

In [ ]:
obs = env.reset()
obs0 = Obs(*obs)
print(obs0)

### Действия

<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p>№</p></th>
<th class="head"><p>Действие</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>0</p></td>
<td><p>Двинуть тележку налево</p></td>
</tr>
<tr class="row-odd"><td><p>1</p></td>
<td><p>Двинуть тележку направо</p></td>
</tr>
</tbody>
</table>

In [ ]:
env.action_space

### Действие  $a = 1$

In [ ]:
obs, reward, done, info = env.step(1)
obs1 = Obs(*obs)
print(obs1)

In [ ]:
print(obs1.v > obs0.v, obs1.omega < obs0.omega)

### Действие $a = 0$

In [ ]:
obs, reward, done, info = env.step(0)
obs2 = Obs(*obs)
print(obs2)

In [ ]:
print(obs2.v < obs1.v, obs2.omega > obs1.omega)

### Терминальные условия

Эпизод заканчивается привыполнении одного из следующих условий:

- Termination: Маятник отклонен от вертикали больше чем на ±12°

- Termination: Положение тележки ±2.4 (центр тележки выходит за границы рисунка)

- Truncation: Эпизод продолжается 500 шагов (200 для версии v0)

### Вознаграждение

Поскольку цель задачи - балансировать маятник около верхнего состояния, вознаграждение +1 назначается за каждый шаг, пока система не достигла терминального состояния.

In [ ]:
def plot_environment(env, figsize=(5,4)):
    plt.figure(figsize=figsize)
    img = env.render(mode="rgb_array")
    plt.imshow(img)
    plt.axis("off")
    return img

In [ ]:
img = plot_environment(env)
plt.show()

### Пример класса для работы с Cartpole

In [ ]:
class CartPolePlayer:
    
    def record_scenario(self, env, policy, num_frames=100) -> dict:
        frames = []
        obs_mat = np.empty((num_frames, 4))
        actions = np.empty((num_frames,))
        rewards = np.empty((num_frames,))
        dones = np.empty((num_frames,), dtype=int)
        first_done_info = ''
        obs = env.reset()  # initial observation
    
        for i in range(num_frames):
            action = policy(obs)
            obs, reward, done, info = env.step(action)
            img = env.render(mode="rgb_array")
            frames.append(img)
            obs_mat[i,:] = obs
            actions[i] = action
            rewards[i] = reward
            dones[i] = int(done)
        
            if done and first_done_info == '':
                first_done_info = info

        record = {'frames': frames, 'obs': obs_mat, 'actions': actions, 'rewards': 
              rewards, 'dones': dones, 'first_done_info':first_done_info}
        
        return record

    
    def update_scene(self, num, frames, patch, time_text, obs_mat, actions, cum_rewards, dones):
        patch.set_data(frames[num])
        text = f"frame: {num}"
        text += ", Obs: ({:.3f}, {:.3f}, {:.3f}, {:.3f})\n".format(*obs_mat[num,:])
        text += f"Action: {actions[num]}"
        text += f", Cumulative Reward: {cum_rewards[num]}"
        text += f", Done: {dones[num]}"
        time_text.set_text(text)
        
        return patch, time_text
    
    
    def plot_animation(self, record, repeat=False, interval=40):
        '''record should contain
        frames: list of N frames
        obs: (N, 4) array of observations
        actions: (N, ) array of actions {0, 1}
        rewards: (N, ) array of rewards at each step {0, 1}
        dones: (N, 1) array of dones {0, 1}
        '''
        cum_rewards = np.cumsum(record['rewards'])
        frames = record['frames']
        fig = plt.figure()
        patch = plt.imshow(record['frames'][0])
        ax = plt.gca()
        time_text = ax.text(0., 0.95,'',horizontalalignment='left',verticalalignment='top', transform=ax.transAxes)
        plt.axis('off')
        anim = animation.FuncAnimation(
            fig, self.update_scene, fargs=(frames, patch, time_text, record['obs'], record['actions'], cum_rewards, record['dones']),
            frames=len(frames), repeat=repeat, interval=interval)
        plt.close()
        
        return anim   
    
    
    def test_policy(self, policy_func, n_scenario = 1000, max_actions = 500, verbose=False):
        final_rewards = []
        for episode in range(n_scenario):
            if verbose and episode % 50 == 0:
                print(episode)
            episode_rewards = 0
            obs = env.reset()  # reset to a random position
            for step in range(max_actions):
                action = policy_func(obs)
                obs, reward, done, info = env.step(action)
                episode_rewards += reward
                if done:
                    break
            final_rewards.append(episode_rewards)
    
        return final_rewards

    
    def plot_policy(self, final_rewards, policy_name:str=''):
        fig = plt.plot(range(len(final_rewards)), final_rewards)
        plt.grid()
        plt.title(policy_name + " Mean Reward {:.2f}, Std Reward {:.2f}".format(np.mean(final_rewards), np.min(final_rewards)))
        plt.ylabel('Cum Reward')
        plt.xlabel('Iteration')
        plt.ylim(0, max(final_rewards) * 1.1)
        
        return fig
    
    
    def save_gif(self, recorded_episode):
        recorded_episode['frames'][1].shape
        write_gif([np.transpose(f, axes=[2,0, 1]) for f in recorded_episode['frames']], 'recorded_episode.gif', fps=30)

In [ ]:
player = CartPolePlayer()

### Стратегия случайных действий

In [ ]:
def rand_policy(obs):
    return random.randint(0, 1)

env.seed(42)
random.seed(0)

rand_rewards = player.test_policy(rand_policy)
pt = player.plot_policy(rand_rewards, "Random Policy")

### Наивная стратегия (управление по отклонению маятника)

In [ ]:
def theta_policy(obs):
    angle = obs[2]
    return 0 if angle < 0 else 1

env.seed(42)
random.seed(0)

theta_rewards = player.test_policy(theta_policy)
pt = player.plot_policy(theta_rewards, "Theta Policy")

### Наивная стратегия (управление по угловой скорости)

In [ ]:
def omega_policy(obs):
    w = obs[3]
    return 1 if w > 0 else 0

env.seed(42)
random.seed(0)

omega_rewards = player.test_policy(omega_policy)
pt = player.plot_policy(omega_rewards, "Omega Policy")

### Комбинация двух наивных стратегий, внезапно приводящая к успеху

In [ ]:
def theta_omega_policy(obs):
    theta, w = obs[2:4]
    if abs(theta) < 0.03:
        return 0 if w < 0 else 1
    else:
        return 0 if theta < 0 else 1
    
env.seed(42)
random.seed(0)

# the cart-pole experiment will end if it lasts more than 500 steps, with info="'TimeLimit.truncated': True"
theta_omega_rewards = player.test_policy(theta_omega_policy, max_actions=510) 
player.plot_policy(theta_omega_rewards, "Theta-Omega Policy")

### Запись эпизода

In [ ]:
theta_record = player.record_scenario(env, theta_omega_policy, 100)

### Анимация

In [ ]:
player.plot_animation(theta_record)

### Сохранение gif

In [ ]:
player.save_gif(theta_record)

# строка для маркдаун - <img src="recorded_episode.gif" width="750" align="center">

In [27]:
class Q_Learning:
    ###########################################################################
    #   START - __init__ function
    ###########################################################################
    # INPUTS: 
    # env - Cart Pole environment
    # alpha - step size 
    # gamma - discount rate
    # epsilon - parameter for epsilon-greedy approach
    # numberEpisodes - total number of simulation episodes
     
    # numberOfBins - this is a 4 dimensional list that defines the number of grid points 
    # for state discretization
    # that is, this list contains number of bins for every state entry, 
    # we have 4 entries, that is,
    # discretization for cart position, cart velocity, pole angle, and pole angular velocity
     
    # lowerBounds - lower bounds (limits) for discretization, list with 4 entries:
    # lower bounds on cart position, cart velocity, pole angle, and pole angular velocity
 
    # upperBounds - upper bounds (limits) for discretization, list with 4 entries:
    # upper bounds on cart position, cart velocity, pole angle, and pole angular velocity
     
    def __init__(self, env, alpha,gamma, epsilon, episodesCount, binCounts, lowerBounds, upperBounds):
        import numpy as np
         
        self.env = env
        self.alpha = alpha
        self.gamma = gamma 
        self.epsilon = epsilon 
        self.actionsCount = env.action_space.n 
        self.episodesCount = episodesCount
        self.binCounts = binCounts
        self.lowerBounds = lowerBounds
        self.upperBounds = upperBounds
         
        # this list stores sum of rewards in every learning episode
        self.sumRewardsEpisode = []
         
        # this matrix is the action value function matrix 
        binSz = (binCounts[0], binCounts[1], binCounts[2], binCounts[3], self.actionsCount)
        self.Qmatrix = np.random.uniform(low=0, high=1, size=binSz)
        

    ###########################################################################
    # START: function "calcStateIdx"
    # for the given 4-dimensional state, and discretization grid defined by 
    # numberOfBins, lowerBounds, and upperBounds, this function will return 
    # the index tuple (4-dimensional) that is used to index entries of the 
    # of the QvalueMatrix 
 
 
    # INPUTS:
    # state - state list/array, 4 entries: 
    # cart position, cart velocity, pole angle, and pole angular velocity
 
    # OUTPUT: 4-dimensional tuple defining the indices of the QvalueMatrix 
    # that correspond to "state" input
 
    ###############################################################################
    def calcStateIdx(self, state):
        position        = state[0]
        velocity        = state[1]
        angle           = state[2]
        angularVelocity = state[3]
         
        cartPositionBin = np.linspace(self.lowerBounds[0], self.upperBounds[0], self.binCounts[0])
        cartVelocityBin = np.linspace(self.lowerBounds[1], self.upperBounds[1], self.binCounts[1])
        poleAngleBin    = np.linspace(self.lowerBounds[2], self.upperBounds[2], self.binCounts[2])
        poleAngleVelocityBin = np.linspace(self.lowerBounds[3], self.upperBounds[3], self.binCounts[3])
         
        positionIdx        = np.maximum(np.digitize(state[0], cartPositionBin) - 1, 0)
        velocityIdx        = np.maximum(np.digitize(state[1], cartVelocityBin) - 1, 0)
        angleIdx           = np.maximum(np.digitize(state[2], poleAngleBin) - 1, 0)
        angularVelocityIdx = np.maximum(np.digitize(state[3], poleAngleVelocityBin) - 1, 0)
         
        return tuple([positionIdx, velocityIdx, angleIdx, angularVelocityIdx])   

        
    ###########################################################################
    #    START - function for selecting an action: epsilon-greedy approach
    ###########################################################################
    # this function selects an action on the basis of the current state 
    # INPUTS: 
    # state - state for which to compute the action
    # index - index of the current episode
    def selectAction(self, state, index):
         
        # first 500 episodes we select completely random actions to have enough exploration
        if index < 500:
            return np.random.choice(self.actionsCount)   
             
        # Returns a random real number in the half-open interval [0.0, 1.0)
        # this number is used for the epsilon greedy approach
        randomNumber = np.random.random()
         
        # after 7000 episodes, we slowly start to decrease the epsilon parameter
        if index > 7000:
            self.epsilon = 0.999 * self.epsilon
         
        # if this condition is satisfied, we are exploring, that is, we select random actions
        if randomNumber < self.epsilon:
            # returns a random action selected from: 0,1,...,actionNumber-1
            return np.random.choice(self.actionsCount)            
         
        # otherwise, we are selecting greedy actions
        else:
            # we return the index where Qmatrix[state,:] has the max value
            # that is, since the index denotes an action, we select greedy actions
            return np.random.choice(np.where(self.Qmatrix[self.calcStateIdx(state)] == np.max(self.Qmatrix[self.calcStateIdx(state)]))[0])
            # here we need to return the minimum index since it can happen
            # that there are several identical maximal entries, for example 
            # import numpy as np
            # a=[0,1,1,0]
            # np.where(a==np.max(a))
            # this will return [1,2], but we only need a single index
            # that is why we need to have np.random.choice(np.where(a==np.max(a))[0])
            # note that zero has to be added here since np.where() returns a tuple
     
     
    ###########################################################################
    #    START - function for simulating learning episodes
    ###########################################################################      
    def simulateEpisodes(self):
        # here we loop through the episodes
        for episodeIdx in range(self.episodesCount):
             
            # list that stores rewards per episode - this is necessary for keeping track of convergence 
            episodeRewards = []
             
            # reset the environment at the beginning of every episode
            current_state = self.env.reset()
            current_state = list(current_state) # s
           
            print("Simulating episode {}".format(episodeIdx))             
             
            # here we step from one state to another
            # this will loop until a terminal state is reached
            terminal_state = False
            while not terminal_state:
                # return a discretized index of the state
                 
                current_state_idx = self.calcStateIdx(current_state)
                 
                # select an action on the basis of the current state, denoted by stateS
                current_action = self.selectAction(current_state, episodeIdx) #  a
                                  
                # here we step and return the state, reward, and boolean denoting if the state is a terminal state
                # prime means that it is the next state
                (next_state, reward, terminal_state, _) = self.env.step(current_action)          
                 
                episodeRewards.append(reward) # r
                 
                next_state = list(next_state) # s'
                 
                next_state_idx = self.calcStateIdx(next_state)
                 
                # return the max value, we do not need actionAprime...
                Qmax_next = np.max(self.Qmatrix[next_state_idx])
                                              
                if not terminal_state:
                    # stateS+(actionA,) - we use this notation to append the tuples
                    # for example, for current_state = (0, 0, 0, 1) and current_action = (1, 0)
                    # we have current_state + (current_action,) = (0, 0, 0, 1, 0)
                    q_update = reward + self.gamma * Qmax_next - self.Qmatrix[current_state_idx + (current_action,)]
                    self.Qmatrix[current_state_idx + (current_action,)] = self.Qmatrix[current_state_idx + (current_action,)] + self.alpha * q_update
                else:
                    # in the terminal state, we have Qmatrix[next_state, next_action] = 0 
                    q_update = reward - self.Qmatrix[current_state_idx + (current_action,)]
                    self.Qmatrix[current_state_idx + (current_action,)] = self.Qmatrix[current_state_idx + (current_action,)] + self.alpha * q_update
                 
                # set the current state to the next state                    
                current_state = next_state
         
            print("Sum of rewards {}".format(np.sum(episodeRewards)))        
            self.sumRewardsEpisode.append(np.sum(episodeRewards))
     
     
    ###########################################################################
    #    START - function for simulating the final learned optimal policy
    ###########################################################################
    # OUTPUT: 
    # env1 - created Cart Pole environment
    # obtainedRewards - a list of obtained rewards during time steps of a single episode
     
    # simulate the final learned optimal policy
    def simulateLearnedStrategy(self):
        cartpole_env = gym.make('CartPole-v1',render_mode = 'human')
        state = cartpole_env.reset()
        cartpole_env.render()
        time_steps = 1000
        
        # obtained rewards at every time step
        obtained_rewards = []
         
        for timeIdx in range(time_steps):
            print(timeIdx)
            
            # select greedy actions
            action = np.random.choice(np.where(self.Qmatrix[self.calcStateIdx(state)] == np.max(self.Qmatrix[self.calcSateIdx(state)]))[0])
            state, reward, terminated, truncated, info = cartpole_env.step(action)
            obtained_rewards.append(reward)
            
            time.sleep(0.05)
            if (terminated):
                time.sleep(1)
                break

        return obtained_rewards, cartpole_env
    
    
    ###########################################################################
    #    START - function for simulating random actions many times
    #   this is used to evaluate the optimal policy and to compare it with a random policy
    ###########################################################################
    #  OUTPUT:
    # sumRewardsEpisodes - every entry of this list is a sum of rewards obtained by simulating the corresponding episode
    # env2 - created Cart Pole environment
    def simulateRandomStrategy(self):
        cartpole_env = gym.make('CartPole-v1')
        state = cartpole_env.reset()
        cartpole_env.render()
        # number of simulation episodes
        episodes_count = 100
        # time steps in every episode
        time_steps = 1000
        # sum of rewards in each episode
        episodes_cum_rewards = []         
         
        for episode_idx in range(episodes_count):
            episode_rewards = []
            initial_state = cartpole_env.reset()
            print(episodeIndex)

            for timeIndex in range(time_steps):
                random_action=cartpole_env.action_space.sample()
                observation, reward, terminated, truncated, info = cartpole_env.step(random_action)
                episode_rewards.append(reward)

                if (terminated):
                    break

            episodes_cum_rewards.append(np.sum(episode_rewards))
            
        return episodes_cum_rewards, cartpole_env 

In [ ]:
env = gym.make('CartPole-v1')
state = env.reset()
 
# here define the parameters for state discretization
upperBounds = env.observation_space.high
lowerBounds = env.observation_space.low
cartVelocityMin = -3
cartVelocityMax = 3
poleAngleVelocityMin = -10
poleAngleVelocityMax = 10
upperBounds[1] = cartVelocityMax
upperBounds[3] = poleAngleVelocityMax
lowerBounds[1] = cartVelocityMin
lowerBounds[3] = poleAngleVelocityMin
 
numberOfBinsPosition = 30
numberOfBinsVelocity = 30
numberOfBinsAngle = 30
numberOfBinsAngleVelocity = 30
numberOfBins = [numberOfBinsPosition, numberOfBinsVelocity, numberOfBinsAngle, numberOfBinsAngleVelocity]
 
# define the parameters
alpha = 0.1
gamma = 1
epsilon = 0.2
numberEpisodes = 15000
 
# create an object
Q1 = Q_Learning(env, alpha, gamma, epsilon, numberEpisodes, numberOfBins, lowerBounds, upperBounds)
# run the Q-Learning algorithm
Q1.simulateEpisodes()
# simulate the learned strategy
(obtainedRewardsOptimal, env1) = Q1.simulateLearnedStrategy()
 
plt.figure(figsize=(12, 5))

# plot the figure and adjust the plot parameters
plt.plot(Q1.sumRewardsEpisode, color='blue', linewidth=1)
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.yscale('log')
plt.show()
plt.savefig('convergence.png')
 
# close the environment
env1.close()
# get the sum of rewards
np.sum(obtainedRewardsOptimal)
 
# now simulate a random strategy
(obtainedRewardsRandom, env2) = Q1.simulateRandomStrategy()
plt.hist(obtainedRewardsRandom)
plt.xlabel('Sum of rewards')
plt.ylabel('Percentage')
plt.savefig('histogram.png')
plt.show()
 
# run this several times and compare with a random learning strategy
(obtainedRewardsOptimal, env1) = Q1.simulateLearnedStrategy()

Simulating episode 0
Sum of rewards 13.0
Simulating episode 1
Sum of rewards 26.0
Simulating episode 2
Sum of rewards 16.0
Simulating episode 3
Sum of rewards 25.0
Simulating episode 4
Sum of rewards 31.0
Simulating episode 5
Sum of rewards 41.0
Simulating episode 6
Sum of rewards 45.0
Simulating episode 7
Sum of rewards 35.0
Simulating episode 8
Sum of rewards 17.0
Simulating episode 9
Sum of rewards 14.0
Simulating episode 10
Sum of rewards 11.0
Simulating episode 11
Sum of rewards 18.0
Simulating episode 12
Sum of rewards 30.0
Simulating episode 13
Sum of rewards 18.0
Simulating episode 14
Sum of rewards 21.0
Simulating episode 15
Sum of rewards 17.0
Simulating episode 16
Sum of rewards 21.0
Simulating episode 17
Sum of rewards 12.0
Simulating episode 18
Sum of rewards 13.0
Simulating episode 19
Sum of rewards 10.0
Simulating episode 20
Sum of rewards 20.0
Simulating episode 21
Sum of rewards 35.0
Simulating episode 22
Sum of rewards 16.0
Simulating episode 23
Sum of rewards 21.0
Si

Sum of rewards 26.0
Simulating episode 213
Sum of rewards 15.0
Simulating episode 214
Sum of rewards 14.0
Simulating episode 215
Sum of rewards 16.0
Simulating episode 216
Sum of rewards 28.0
Simulating episode 217
Sum of rewards 17.0
Simulating episode 218
Sum of rewards 14.0
Simulating episode 219
Sum of rewards 35.0
Simulating episode 220
Sum of rewards 18.0
Simulating episode 221
Sum of rewards 13.0
Simulating episode 222
Sum of rewards 14.0
Simulating episode 223
Sum of rewards 21.0
Simulating episode 224
Sum of rewards 15.0
Simulating episode 225
Sum of rewards 28.0
Simulating episode 226
Sum of rewards 17.0
Simulating episode 227
Sum of rewards 22.0
Simulating episode 228
Sum of rewards 13.0
Simulating episode 229
Sum of rewards 29.0
Simulating episode 230
Sum of rewards 13.0
Simulating episode 231
Sum of rewards 28.0
Simulating episode 232
Sum of rewards 27.0
Simulating episode 233
Sum of rewards 15.0
Simulating episode 234
Sum of rewards 10.0
Simulating episode 235
Sum of rewa

Sum of rewards 22.0
Simulating episode 417
Sum of rewards 17.0
Simulating episode 418
Sum of rewards 17.0
Simulating episode 419
Sum of rewards 26.0
Simulating episode 420
Sum of rewards 16.0
Simulating episode 421
Sum of rewards 14.0
Simulating episode 422
Sum of rewards 24.0
Simulating episode 423
Sum of rewards 18.0
Simulating episode 424
Sum of rewards 19.0
Simulating episode 425
Sum of rewards 17.0
Simulating episode 426
Sum of rewards 11.0
Simulating episode 427
Sum of rewards 25.0
Simulating episode 428
Sum of rewards 21.0
Simulating episode 429
Sum of rewards 13.0
Simulating episode 430
Sum of rewards 22.0
Simulating episode 431
Sum of rewards 14.0
Simulating episode 432
Sum of rewards 12.0
Simulating episode 433
Sum of rewards 22.0
Simulating episode 434
Sum of rewards 20.0
Simulating episode 435
Sum of rewards 28.0
Simulating episode 436
Sum of rewards 14.0
Simulating episode 437
Sum of rewards 14.0
Simulating episode 438
Sum of rewards 36.0
Simulating episode 439
Sum of rewa

Sum of rewards 119.0
Simulating episode 608
Sum of rewards 87.0
Simulating episode 609
Sum of rewards 127.0
Simulating episode 610
Sum of rewards 95.0
Simulating episode 611
Sum of rewards 112.0
Simulating episode 612
Sum of rewards 63.0
Simulating episode 613
Sum of rewards 218.0
Simulating episode 614
Sum of rewards 23.0
Simulating episode 615
Sum of rewards 107.0
Simulating episode 616
Sum of rewards 82.0
Simulating episode 617
Sum of rewards 112.0
Simulating episode 618
Sum of rewards 147.0
Simulating episode 619
Sum of rewards 71.0
Simulating episode 620
Sum of rewards 201.0
Simulating episode 621
Sum of rewards 53.0
Simulating episode 622
Sum of rewards 85.0
Simulating episode 623
Sum of rewards 88.0
Simulating episode 624
Sum of rewards 133.0
Simulating episode 625
Sum of rewards 118.0
Simulating episode 626
Sum of rewards 113.0
Simulating episode 627
Sum of rewards 151.0
Simulating episode 628
Sum of rewards 27.0
Simulating episode 629
Sum of rewards 160.0
Simulating episode 63

Sum of rewards 261.0
Simulating episode 799
Sum of rewards 111.0
Simulating episode 800
Sum of rewards 239.0
Simulating episode 801
Sum of rewards 82.0
Simulating episode 802
Sum of rewards 117.0
Simulating episode 803
Sum of rewards 72.0
Simulating episode 804
Sum of rewards 54.0
Simulating episode 805
Sum of rewards 191.0
Simulating episode 806
Sum of rewards 19.0
Simulating episode 807
Sum of rewards 104.0
Simulating episode 808
Sum of rewards 64.0
Simulating episode 809
Sum of rewards 22.0
Simulating episode 810
Sum of rewards 175.0
Simulating episode 811
Sum of rewards 316.0
Simulating episode 812
Sum of rewards 86.0
Simulating episode 813
Sum of rewards 139.0
Simulating episode 814
Sum of rewards 164.0
Simulating episode 815
Sum of rewards 120.0
Simulating episode 816
Sum of rewards 115.0
Simulating episode 817
Sum of rewards 263.0
Simulating episode 818
Sum of rewards 110.0
Simulating episode 819
Sum of rewards 83.0
Simulating episode 820
Sum of rewards 129.0
Simulating episode 

...

Sum of rewards 123.0
Simulating episode 6894
Sum of rewards 193.0
Simulating episode 6895
Sum of rewards 178.0
Simulating episode 6896
Sum of rewards 199.0
Simulating episode 6897
Sum of rewards 113.0
Simulating episode 6898
Sum of rewards 229.0
Simulating episode 6899
Sum of rewards 120.0
Simulating episode 6900
Sum of rewards 32.0
Simulating episode 6901
Sum of rewards 155.0
Simulating episode 6902
Sum of rewards 155.0
Simulating episode 6903
Sum of rewards 100.0
Simulating episode 6904
Sum of rewards 129.0
Simulating episode 6905
Sum of rewards 330.0
Simulating episode 6906
Sum of rewards 225.0
Simulating episode 6907
Sum of rewards 153.0
Simulating episode 6908
Sum of rewards 167.0
Simulating episode 6909
Sum of rewards 330.0
Simulating episode 6910
Sum of rewards 265.0
Simulating episode 6911
Sum of rewards 180.0
Simulating episode 6912
Sum of rewards 145.0
Simulating episode 6913
Sum of rewards 62.0
Simulating episode 6914
Sum of rewards 138.0
Simulating episode 6915
Sum of rewar

Sum of rewards 162.0
Simulating episode 7078
Sum of rewards 143.0
Simulating episode 7079
Sum of rewards 123.0
Simulating episode 7080
Sum of rewards 132.0
Simulating episode 7081
Sum of rewards 114.0
Simulating episode 7082
Sum of rewards 153.0
Simulating episode 7083
Sum of rewards 114.0
Simulating episode 7084
Sum of rewards 158.0
Simulating episode 7085
Sum of rewards 212.0
Simulating episode 7086
Sum of rewards 163.0
Simulating episode 7087
Sum of rewards 147.0
Simulating episode 7088
Sum of rewards 180.0
Simulating episode 7089
Sum of rewards 156.0
Simulating episode 7090
Sum of rewards 175.0
Simulating episode 7091
Sum of rewards 132.0
Simulating episode 7092
Sum of rewards 155.0
Simulating episode 7093
Sum of rewards 148.0
Simulating episode 7094
Sum of rewards 201.0
Simulating episode 7095
Sum of rewards 185.0
Simulating episode 7096
Sum of rewards 179.0
Simulating episode 7097
Sum of rewards 184.0
Simulating episode 7098
Sum of rewards 163.0
Simulating episode 7099
Sum of rew

Sum of rewards 212.0
Simulating episode 7261
Sum of rewards 199.0
Simulating episode 7262
Sum of rewards 163.0
Simulating episode 7263
Sum of rewards 159.0
Simulating episode 7264
Sum of rewards 131.0
Simulating episode 7265
Sum of rewards 144.0
Simulating episode 7266
Sum of rewards 123.0
Simulating episode 7267
Sum of rewards 131.0
Simulating episode 7268
Sum of rewards 150.0
Simulating episode 7269
Sum of rewards 129.0
Simulating episode 7270
Sum of rewards 102.0
Simulating episode 7271
Sum of rewards 86.0
Simulating episode 7272
Sum of rewards 121.0
Simulating episode 7273
Sum of rewards 172.0
Simulating episode 7274
Sum of rewards 165.0
Simulating episode 7275
Sum of rewards 216.0
Simulating episode 7276
Sum of rewards 256.0
Simulating episode 7277
Sum of rewards 168.0
Simulating episode 7278
Sum of rewards 151.0
Simulating episode 7279
Sum of rewards 157.0
Simulating episode 7280
Sum of rewards 188.0
Simulating episode 7281
Sum of rewards 308.0
Simulating episode 7282
Sum of rewa

Sum of rewards 126.0
Simulating episode 7444
Sum of rewards 140.0
Simulating episode 7445
Sum of rewards 106.0
Simulating episode 7446
Sum of rewards 77.0
Simulating episode 7447
Sum of rewards 82.0
Simulating episode 7448
Sum of rewards 149.0
Simulating episode 7449
Sum of rewards 133.0
Simulating episode 7450
Sum of rewards 105.0
Simulating episode 7451
Sum of rewards 216.0
Simulating episode 7452
Sum of rewards 266.0
Simulating episode 7453
Sum of rewards 203.0
Simulating episode 7454
Sum of rewards 217.0
Simulating episode 7455
Sum of rewards 234.0
Simulating episode 7456
Sum of rewards 130.0
Simulating episode 7457
Sum of rewards 163.0
Simulating episode 7458
Sum of rewards 166.0
Simulating episode 7459
Sum of rewards 159.0
Simulating episode 7460
Sum of rewards 145.0
Simulating episode 7461
Sum of rewards 160.0
Simulating episode 7462
Sum of rewards 192.0
Simulating episode 7463
Sum of rewards 257.0
Simulating episode 7464
Sum of rewards 159.0
Simulating episode 7465
Sum of rewar

Sum of rewards 136.0
Simulating episode 7628
Sum of rewards 126.0
Simulating episode 7629
Sum of rewards 193.0
Simulating episode 7630
Sum of rewards 73.0
Simulating episode 7631
Sum of rewards 168.0
Simulating episode 7632
Sum of rewards 156.0
Simulating episode 7633
Sum of rewards 138.0
Simulating episode 7634
Sum of rewards 159.0
Simulating episode 7635
Sum of rewards 175.0
Simulating episode 7636
Sum of rewards 104.0
Simulating episode 7637
Sum of rewards 160.0
Simulating episode 7638
Sum of rewards 154.0
Simulating episode 7639
Sum of rewards 179.0
Simulating episode 7640
Sum of rewards 185.0
Simulating episode 7641
Sum of rewards 170.0
Simulating episode 7642
Sum of rewards 153.0
Simulating episode 7643
Sum of rewards 143.0
Simulating episode 7644
Sum of rewards 183.0
Simulating episode 7645
Sum of rewards 174.0
Simulating episode 7646
Sum of rewards 147.0
Simulating episode 7647
Sum of rewards 139.0
Simulating episode 7648
Sum of rewards 155.0
Simulating episode 7649
Sum of rewa